# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [83]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag, ne_chunk
from nltk.corpus import wordnet

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/antelinvestigacionydesarrollo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [84]:
# load data from database
DB_NAME = 'disaster.db'
MESSAGES_TABLE = 'messages'

engine = create_engine('sqlite:///{}'.format(DB_NAME))
df = pd.read_sql_table(MESSAGES_TABLE, engine)
print(df.shape)
df.head()

(26180, 41)


,index,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
df.isnull().sum()

index                         0
id                            0
message                       0
original                  16027
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
child_alone                   0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
related                       0
request 

In [86]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [23]:
# Let's keep only the English version of the messages, as the final classifier
# is based only in English messages
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

print(X.shape)
print(y.shape)

(26180,)
(26180, 36)


In [24]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [25]:
y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [26]:
text = X[3]
text

'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.'

In [27]:
def get_wordnet_pos(treebank_tag):
    """ 
    Transforms from Treebank tags to wordnet tags.
    As discussed here: 
    https://stackoverflow.com/questions/15586721/
    wordnet-lemmatization-and-pos-tagging-in-python
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # If unknown, return the default value

In [28]:
# Case normalization
temp_text = text.lower()

# Punctuation removal
temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

# Tokenize
tokens = word_tokenize(temp_text)

# Stop Word Removal
stop_words = stopwords.words("english")
tokens = [word for word in tokens if word not in stop_words]

# Part-of-Speech Tagging
tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

# Named Entity Recognition
# TODO: Add this to the pipeline. The punctuation is important to recognize the
# entities.

# Lemmatization
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(*token) for token in tokens]

# Stemming
stemmer = PorterStemmer()
tokens = [stemmer.stem(word) for word in tokens]

In [29]:
tokens

['un',
 'report',
 'leogan',
 '80',
 '90',
 'destroy',
 'hospit',
 'st',
 'croix',
 'function',
 'need',
 'suppli',
 'desper']

In [49]:
def tokenize(text):
    """ Basic tokenization function. """
    # Case normalization
    temp_text = text.lower()

    # Punctuation removal
    temp_text = re.sub(r'[^a-zA-Z0-9]', ' ', temp_text)

    # Tokenize
    tokens = word_tokenize(temp_text)

    # Stop Word Removal
    stop_words = stopwords.words("english")
    tokens = [word for word in tokens if word not in stop_words]

    # Part-of-Speech Tagging
    tokens = [(token[0], get_wordnet_pos(token[1])) for token in pos_tag(tokens)]

    # Named Entity Recognition
    # TODO: Add this to the pipeline. The punctuation is important to recognize the
    # entities.

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(*token) for token in tokens]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens

In [50]:
%time tokenize(X[0])

CPU times: user 3.81 ms, sys: 1.73 ms, total: 5.54 ms
Wall time: 4.03 ms


['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']

In [47]:
def time_tokenize(X):
    result = list()
    for text in X[:10]:
        result.append(tokenize(text))
    return result

In [51]:
%timeit time_tokenize(X)

14.6 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
'Estimated tokenization time: {} seconds'.format((14.6 * 26200 / 10) / 1000)

'Estimated tokenization time: 38.252 seconds'

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [34]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [35]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x10a172620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [53]:
vect = CountVectorizer(tokenizer=tokenize)
%time X_v = vect.fit_transform(X)

CPU times: user 1min 2s, sys: 3.22 s, total: 1min 5s
Wall time: 1min 6s


In [63]:
X_v[:10].sum(axis=1)

matrix([[ 8],
        [ 1],
        [ 3],
        [13],
        [ 8],
        [ 3],
        [ 4],
        [ 6],
        [ 5],
        [12]])

In [64]:
import numpy as np

In [67]:
df_v = pd.DataFrame(X_v.todense())

In [71]:
df_v.head()

,0,1,2,3,4,5,6,7,8,9,...,26308,26309,26310,26311,26312,26313,26314,26315,26316,26317
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
%time df_v.isnull().sum().sum()

CPU times: user 10.1 s, sys: 5.34 s, total: 15.5 s
Wall time: 16.7 s


0

In [70]:
tfidf = TfidfTransformer()

In [73]:
%time X_t = tfidf.fit_transform(df_v)

CPU times: user 10.8 s, sys: 2.25 s, total: 13 s
Wall time: 14 s


In [77]:
df_t = pd.DataFrame(X_t.todense())
df_t.head()

,0,1,2,3,4,5,6,7,8,9,...,26308,26309,26310,26311,26312,26313,26314,26315,26316,26317
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df_t.isnull().sum().sum()

0

In [80]:
y.isnull().sum().sum()

7416

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)
pipeline.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.